PART I

In [179]:
import re
import copy
from functools import reduce
from itertools import combinations

In [236]:
input = "input.txt"
with open(input, 'r') as infile:
    workflows = {}
    parts = []
    workflows_section = True
    part_regex = re.compile('\{x=(\d+),m=(\d+),a=(\d+),s=(\d+)\}')
    condition_regex = re.compile('([xmas])([<>])(\d+):([ARa-z]+)')
    for l in infile.readlines():
        if not l.strip():
            workflows_section = False
            continue
        if workflows_section:
            workflow_raw = l.strip()
            workflow_name_match = re.match('(\w+)\{', workflow_raw)
            workflow_name = workflow_name_match.groups()[0]            
            workflow_split = workflow_raw[workflow_name_match.end():].split(',')
            condition_list = []
            for condition in workflow_split[:-1]:
                #comparee is probably not a word
                category, comparator, comparee, target =  re.match(condition_regex, condition).groups()
                condition_list.append([category, comparator, int(comparee), target])
            condition_list.append(workflow_split[-1][:-1])
            workflows[workflow_name] = condition_list
        else:
            x,m,a,s = re.fullmatch(part_regex, l.strip()).groups()
            parts.append({'x':int(x), 'm':int(m), 'a':int(a), 's':int(s)})


In [127]:
def process_part(part, current_workflow):
    if current_workflow == 'A':
        return True
    elif current_workflow == 'R':
        return False
    conditions = workflows[current_workflow]
    for condition in conditions[:-1]:
        if condition[1] == '<':
            if part[condition[0]] < condition[2]:
                return process_part(part, condition[3])
            else:
                continue
        else:
            if part[condition[0]] > condition[2]:
                return process_part(part, condition[3])
            else:
                continue
    return process_part(part, conditions[-1])

In [128]:
sum(sum(part.values()) for part in parts if process_part(part, 'in'))

19114

PART II

I only need the workflow for this, the parts I construct procedurally.

In [237]:
workflows

{'fg': [['x', '>', 1250, 'A'], 'R'],
 'tbx': [['x', '>', 3829, 'R'], 'R'],
 'zkd': [['a', '<', 3370, 'tsc'], ['m', '>', 1007, 'vll'], 'A'],
 'skj': [['m', '<', 551, 'A'], ['m', '>', 1174, 'rxr'], 'rm'],
 'kjj': [['s', '>', 1675, 'R'],
  ['s', '>', 1293, 'A'],
  ['x', '>', 1269, 'A'],
  'A'],
 'ljf': [['a', '<', 2823, 'R'], ['s', '>', 1542, 'rgs'], 'hnt'],
 'hkz': [['s', '>', 1809, 'R'], ['s', '<', 1051, 'R'], 'A'],
 'jjf': [['m', '>', 3087, 'R'], ['m', '>', 2948, 'A'], 'A'],
 'hzh': [['s', '<', 2520, 'R'],
  ['x', '<', 2982, 'vpj'],
  ['m', '<', 1064, 'tl'],
  'lpq'],
 'kr': [['x', '>', 2488, 'R'], ['x', '<', 2431, 'A'], 'R'],
 'krn': [['x', '<', 352, 'R'],
  ['m', '<', 3550, 'A'],
  ['s', '>', 1244, 'jkh'],
  'dfg'],
 'bvz': [['a', '<', 2812, 'kr'],
  ['a', '>', 2932, 'cvx'],
  ['a', '<', 2867, 'hdq'],
  'R'],
 'dqh': [['x', '<', 1411, 'khk'],
  ['x', '>', 1928, 'kt'],
  ['a', '>', 2702, 'hm'],
  'xcv'],
 'stz': [['m', '<', 2007, 'A'], 'R'],
 'xsl': [['m', '<', 1405, 'R'], ['x', '>', 

How many 'paths to acceptance' are there?

In [238]:
def create_intial_solution_space():
    return {'x':[1,4000],
            'm':[1,4000],
            'a':[1,4000],
            's':[1,4000],
           }

In [239]:
def reduce_solution_space(solution_space, current_workflow):
    if current_workflow == 'A':
        return [solution_space]
    if current_workflow == 'R':
        return []
    conditions = workflows[current_workflow]
    recursive_solution = []
    for condition in conditions[:-1]:
        #print(condition)
        if condition[1] == '<':
            #Check if the current space can fulfil this
            if solution_space[condition[0]][0] < condition[2]:
                new_solution_space = copy.deepcopy(solution_space)
                new_solution_space[condition[0]][1] = min(new_solution_space[condition[0]][1], condition[2] - 1)
                recursive_solution += reduce_solution_space(new_solution_space, condition[3])
            if solution_space[condition[0]][1] >= condition[2]:
                #We continue with the next condition with a solution space that cannot fulfil the condition!
                solution_space[condition[0]][0] = max(solution_space[condition[0]][0], condition[2])
                
        else:
            if solution_space[condition[0]][1] > condition[2]:
                new_solution_space = copy.deepcopy(solution_space)
                new_solution_space[condition[0]][0] = max(new_solution_space[condition[0]][0], condition[2] + 1)
                recursive_solution += reduce_solution_space(new_solution_space, condition[3])
            if solution_space[condition[0]][0] <= condition[2]:
                solution_space[condition[0]][1] = min(solution_space[condition[0]][1], condition[2])                            
    recursive_solution += reduce_solution_space(solution_space, conditions[-1])
    return recursive_solution

In [240]:
all_solution_spaces = reduce_solution_space(create_intial_solution_space(), 'in')

In [241]:
all_solution_spaces

[{'x': [1, 4000], 'm': [2896, 4000], 'a': [1, 1118], 's': [973, 2009]},
 {'x': [2452, 4000], 'm': [2026, 3102], 'a': [1, 459], 's': [327, 551]},
 {'x': [2452, 3482], 'm': [2026, 3102], 'a': [460, 1118], 's': [327, 551]},
 {'x': [1, 1245], 'm': [2026, 2900], 'a': [1, 563], 's': [728, 972]},
 {'x': [1, 2451], 'm': [2026, 4000], 'a': [564, 1118], 's': [1, 489]},
 {'x': [1, 345], 'm': [2026, 4000], 'a': [564, 1118], 's': [490, 972]},
 {'x': [346, 907], 'm': [3527, 4000], 'a': [564, 1118], 's': [490, 972]},
 {'x': [346, 907], 'm': [2772, 3526], 'a': [564, 1118], 's': [490, 972]},
 {'x': [2626, 3114], 'm': [2026, 2803], 'a': [1119, 2293], 's': [1, 2009]},
 {'x': [2863, 3114], 'm': [3377, 4000], 'a': [1791, 2293], 's': [1, 503]},
 {'x': [2626, 2862], 'm': [3377, 3787], 'a': [1119, 2293], 's': [1, 2009]},
 {'x': [1, 2625], 'm': [2911, 4000], 'a': [1398, 1721], 's': [1, 2009]},
 {'x': [1, 2625], 'm': [2026, 2910], 'a': [1398, 1721], 's': [1243, 2009]},
 {'x': [1, 2625], 'm': [2589, 2910], 'a': 

Subtract the overlaps...

In [242]:
def get_solution_overlap(sol1, sol2):
    overlaps = [max(0, min(range1[1],range2[1]) - max(range1[0],range2[0]) + 1) for range1, range2 in zip(sol1.values(), sol2.values())]
    return reduce(lambda x,y: x*y, overlaps)

In [243]:
all_overlaps = sum(get_solution_overlap(all_solution_spaces[a],all_solution_spaces[b]) for a,b in combinations(range(len(all_solution_spaces)),2))

In [244]:
all_solution_space_ranges = [[(maxval - minval)+1 for (minval, maxval) in space.values()] for space in all_solution_spaces]

In [246]:
(sum(reduce(lambda x,y: x*y, ran) for ran in all_solution_space_ranges) - all_overlaps)

124615747767410

Oh boi <3